In [1]:
1

1

In [2]:
# import pymongo
import datetime

In [3]:
import glob
import pandas as pd
import numpy as np
import os
import joblib
import json
import pickle
import gc
from collections import defaultdict
# from functools import reduce
# import bz2
import gzip
import pathlib
import dask.dataframe as dd
import string

/opt/conda/lib/python3.8/site-packages/dask/dataframe/utils.py:366: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/utils.py:366: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/utils.py:366: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [4]:
import vaex

In [5]:
def d2j(d, f):
    with open(f, mode="wt", encoding="utf-8") as file:
        json.dump(d, file, ensure_ascii=False, indent=2)
    return 0
        
def f2d(f):
    with open(f) as file:
        d = json.load(file)
    return d

In [6]:
def d2jg(d, f):
    with gzip.open(f, mode="wt", encoding="utf-8") as file:
        json.dump(d, file, ensure_ascii=False, indent=2)
    return 0

def fg2d(f):
    with gzip.open(f) as file:
        d = json.load(file)
    return d

In [7]:
# def d2jz(d, f):
#     with bz2.open(f, mode="wt", encoding="utf-8") as file:
#         json.dump(d, file, ensure_ascii=False, indent=2)
        
# def fz2d(f):
#     with bz2.open(f) as file:
#         d = json.load(file)
#     return d

In [8]:
def d2p(d, f):
    with open(f, mode="wb") as handle:
        pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return 0
        
def p2d(f):
    with open(f, "rb") as handle:
        d = pickle.load(handle)
    return d

In [9]:
nw = datetime.datetime.now()
ts = nw.strftime("%Y%m%d_%H%M%S")
# ts = ""
print(ts)

20231003_083542


In [10]:
# inc_list = ["IY", "HO","KO","CD", "SI", "SY", "SB", "IR", "CZ", "SH"]
inc_list = ["iy", "ho", "ko", "cd", "si","to", "sy", "sb", "cz", "sh", "re"]

In [11]:
# f_csv = "../p3_dbcreation/df_csv20230630_083034.pkl"
f_csv = "./df_csv_raw20230828_001100.pkl"
df_csv_ = pd.read_pickle(f_csv)
# for g in df_csv:
#     print(g)
#     break
df_csv_.head()

,f,gp0,gp,file,scheme,tbl,ym
0,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/bu/dpc_bu_201104_...,t/hdw0,dpc_bu_201104_000.csv,dpc,bu,201104
1,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/bu/dpc_bu_201105_...,t/hdw0,dpc_bu_201105_000.csv,dpc,bu,201105
2,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/bu/dpc_bu_201106_...,t/hdw0,dpc_bu_201106_000.csv,dpc,bu,201106
3,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/bu/dpc_bu_201107_...,t/hdw0,dpc_bu_201107_000.csv,dpc,bu,201107
4,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/bu/dpc_bu_201108_...,t/hdw0,dpc_bu_201108_000.csv,dpc,bu,201108


In [12]:
def adc1(l):
    #default exclusion
    dflt_ex = [3,4]
    for e in dflt_ex:        
        if e not in l:
            l.append(e)
    
    l = list(set(l))
    l = sorted(l)
    return ["c" + str(i) for i in l]
#     return ["c" + str(i) for i in l] + ["id1"]

def rn(st, en):
    return [i for i in range(st, en+1)]


d_drop = dict()
#dpc
d_drop[("dpc", "ir")] = adc1(rn(9,13))
d_drop[("dpc", "re")] = adc1(rn(5,9)+rn(14,15)+rn(17, 24)+rn(27, 35)+[38])
d_drop[("dpc", "ho")] = adc1(rn(3,8))
d_drop[("dpc", "ko")] = adc1(rn(3,7))
d_drop[("dpc", "sb")] = adc1(rn(3,5) + [9, 13])
d_drop[("dpc", "sy")] = adc1(rn(3,5) + [11, 13])
d_drop[("dpc", "sk")] = adc1(rn(3,5) + [9, 11])
d_drop[("dpc", "hh")] = adc1(rn(3,5))
d_drop[("dpc", "si")] = adc1(rn(3,5) + rn(12, 48))
d_drop[("dpc", "iy")] = adc1(rn(3,5) + rn(12, 48))
d_drop[("dpc", "to")] = adc1(rn(3,5) + rn(12, 48))
d_drop[("dpc", "cd")] = adc1(rn(3,5) + [15])

#med
d_drop[("med", "ir")] = adc1(rn(9,13))
d_drop[("med", "re")] = adc1(rn(5,9)+rn(14,27)+rn(29, 43)+[47])
d_drop[("med", "ho")] = adc1(rn(3,8))
d_drop[("med", "ko")] = adc1(rn(3,7))

d_drop[("med", "sy")] = adc1(rn(3,5) + [11, 14])

d_drop[("med", "si")] = adc1(rn(3,5) + rn(12, 53))
d_drop[("med", "iy")] = adc1(rn(3,5) + rn(12, 53))
d_drop[("med", "to")] = adc1(rn(3,5) + rn(13, 57))

#pha
d_drop[("pha", "yk")] = adc1(rn(9,12))
d_drop[("pha", "re")] = adc1(rn(5,9)+rn(14,16)+[19]+[21]+[23]+[26]+[29])
d_drop[("pha", "ho")] = adc1(rn(3,8))
d_drop[("pha", "ko")] = adc1(rn(3,7))

d_drop[("pha", "sh")] = adc1(rn(3,5) + [9])
d_drop[("pha", "cz")] = adc1(rn(3,5) + rn(10, 48))
d_drop[("pha", "iy")] = adc1(rn(3,5))
d_drop[("pha", "to")] = adc1(rn(3,5))
d_drop[("pha", "ki")] = adc1(rn(3,5)+rn(8,113))
# d_drop[("pha", "st")] = adc1(rn(3,5))

In [13]:
d_schm = {
    "dpc":[],
    "med":[],
    "pha":[]
}
for k in d_drop.keys():
    d_schm[k[0]].append(k[1])
d_schm

{'dpc': ['ir',
  're',
  'ho',
  'ko',
  'sb',
  'sy',
  'sk',
  'hh',
  'si',
  'iy',
  'to',
  'cd'],
 'med': ['ir', 're', 'ho', 'ko', 'sy', 'si', 'iy', 'to'],
 'pha': ['yk', 're', 'ho', 'ko', 'sh', 'cz', 'iy', 'to', 'ki']}

In [14]:
tag = "dpc"
l_tb = d_schm[tag]
df_dpc = df_csv_\
    .query("scheme == @tag")\
    .query("tbl in @l_tb")
df_dpc.head()

,f,gp0,gp,file,scheme,tbl,ym
132,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_...,t/hdw0,dpc_cd_201104_000.csv,dpc,cd,201104
133,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_...,t/hdw0,dpc_cd_201104_001.csv,dpc,cd,201104
134,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_...,t/hdw0,dpc_cd_201104_002.csv,dpc,cd,201104
135,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/cd/dpc_cd_201105_...,t/hdw0,dpc_cd_201105_000.csv,dpc,cd,201105
136,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/cd/dpc_cd_201105_...,t/hdw0,dpc_cd_201105_001.csv,dpc,cd,201105


In [15]:
tag = "med"
l_tb = d_schm[tag]
df_med = df_csv_\
    .query("scheme == @tag")\
    .query("tbl in @l_tb")

df_med.head()

,f,gp0,gp,file,scheme,tbl,ym
2318,../../../work/dat/dat/hdw050/202209-018/data/m...,t/hdw050/202209-018/data/med/ho/med_ho_201104_...,t/hdw0,med_ho_201104_000.csv,med,ho,201104
2319,../../../work/dat/dat/hdw050/202209-018/data/m...,t/hdw050/202209-018/data/med/ho/med_ho_201105_...,t/hdw0,med_ho_201105_000.csv,med,ho,201105
2320,../../../work/dat/dat/hdw050/202209-018/data/m...,t/hdw050/202209-018/data/med/ho/med_ho_201106_...,t/hdw0,med_ho_201106_000.csv,med,ho,201106
2321,../../../work/dat/dat/hdw050/202209-018/data/m...,t/hdw050/202209-018/data/med/ho/med_ho_201107_...,t/hdw0,med_ho_201107_000.csv,med,ho,201107
2322,../../../work/dat/dat/hdw050/202209-018/data/m...,t/hdw050/202209-018/data/med/ho/med_ho_201108_...,t/hdw0,med_ho_201108_000.csv,med,ho,201108


In [16]:
tag = "pha"
l_tb = d_schm[tag]
df_pha = df_csv_\
    .query("scheme == @tag")\
    .query("tbl in @l_tb")


df_pha.head()

,f,gp0,gp,file,scheme,tbl,ym
4027,../../../work/dat/dat/hdw050/202209-018/data/p...,t/hdw050/202209-018/data/pha/cz/pha_cz_201104_...,t/hdw0,pha_cz_201104_000.csv,pha,cz,201104
4028,../../../work/dat/dat/hdw050/202209-018/data/p...,t/hdw050/202209-018/data/pha/cz/pha_cz_201104_...,t/hdw0,pha_cz_201104_001.csv,pha,cz,201104
4029,../../../work/dat/dat/hdw050/202209-018/data/p...,t/hdw050/202209-018/data/pha/cz/pha_cz_201104_...,t/hdw0,pha_cz_201104_002.csv,pha,cz,201104
4030,../../../work/dat/dat/hdw050/202209-018/data/p...,t/hdw050/202209-018/data/pha/cz/pha_cz_201104_...,t/hdw0,pha_cz_201104_003.csv,pha,cz,201104
4031,../../../work/dat/dat/hdw050/202209-018/data/p...,t/hdw050/202209-018/data/pha/cz/pha_cz_201105_...,t/hdw0,pha_cz_201105_000.csv,pha,cz,201105


In [17]:
df_csv = pd.concat([df_dpc, df_med, df_pha])\
    .sort_values(["scheme", "ym"])

df_csv.shape

(10017, 7)

In [18]:
df_csv.head()

,f,gp0,gp,file,scheme,tbl,ym
132,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_...,t/hdw0,dpc_cd_201104_000.csv,dpc,cd,201104
133,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_...,t/hdw0,dpc_cd_201104_001.csv,dpc,cd,201104
134,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_...,t/hdw0,dpc_cd_201104_002.csv,dpc,cd,201104
734,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/hh/dpc_hh_201104_...,t/hdw0,dpc_hh_201104_000.csv,dpc,hh,201104
866,../../../work/dat/dat/hdw050/202209-018/data/d...,t/hdw050/202209-018/data/dpc/ho/dpc_ho_201104_...,t/hdw0,dpc_ho_201104_000.csv,dpc,ho,201104


In [19]:
"../../../storage/archives/dat/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_000.csv"
"../../../work/dat/dat/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_000.csv"

'../../../work/dat/dat/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_000.csv'

In [20]:
%%time
outdir = "../../../work/procdat/insvaex/"
l_proc = []

for r in df_csv.itertuples():
    d = r._asdict()
    del d["Index"]
    f = r.f
    p = pathlib.Path(f)
    d["odr"] = d["file"][-7:-4]
    d["f"] = d["f"].replace("/work/dat/dat/","/share3/")
    f_out = outdir + p.name.replace("csv", "hdf5")
    f_out_base = p.name.replace("csv", "hdf5")
    d["f_pq"] = f_out
    d["f_pq_base"] = f_out_base
    l_proc.append(d)
#     fn = pathlib.Path(r.f)
#     print(fn)
#     break

CPU times: user 144 ms, sys: 7.98 ms, total: 152 ms
Wall time: 151 ms


In [21]:
l_proc[:3]

[{'f': '../../../share3/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_000.csv',
  'gp0': 't/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_000.csv',
  'gp': 't/hdw0',
  'file': 'dpc_cd_201104_000.csv',
  'scheme': 'dpc',
  'tbl': 'cd',
  'ym': '201104',
  'odr': '000',
  'f_pq': '../../../work/procdat/insvaex/dpc_cd_201104_000.hdf5',
  'f_pq_base': 'dpc_cd_201104_000.hdf5'},
 {'f': '../../../share3/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_001.csv',
  'gp0': 't/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_001.csv',
  'gp': 't/hdw0',
  'file': 'dpc_cd_201104_001.csv',
  'scheme': 'dpc',
  'tbl': 'cd',
  'ym': '201104',
  'odr': '001',
  'f_pq': '../../../work/procdat/insvaex/dpc_cd_201104_001.hdf5',
  'f_pq_base': 'dpc_cd_201104_001.hdf5'},
 {'f': '../../../share3/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_002.csv',
  'gp0': 't/hdw050/202209-018/data/dpc/cd/dpc_cd_201104_002.csv',
  'gp': 't/hdw0',
  'file': 'dpc_cd_201104_002.csv',
  'scheme': 'dpc',
  'tbl': 'cd',
  'ym': '201104',
  'od

In [22]:
f_pkls = "../../../share/procdat/p1_dbcreation/d_dict_fs_dict2_20230626_093330.json"
# f_pkls = "../../../share/procdat/p1_dbcreation_split/d_dict_sub_20230829_053938.json"
d_dict = f2d(f_pkls)

In [23]:
# d_dict

In [24]:
d_id1 = d_dict[r.scheme][r.ym].replace("strage", "share")
d_id1

'../../../share/procdat/p1_dbcreation/d_seq2_to_id1n_pha_202203.pkl'

In [25]:
chrs = string.digits + string.ascii_lowercase[:6]
chrs = [c for c in chrs]
chrs

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f']

In [27]:
for c in chrs:
    d_c = "../../../storage/parquets/d_" +c
    print(d_c)
    !mkdir $d_c

../../../storage/parquets/d_0
../../../storage/parquets/d_1
../../../storage/parquets/d_2
../../../storage/parquets/d_3
../../../storage/parquets/d_4
../../../storage/parquets/d_5
../../../storage/parquets/d_6
../../../storage/parquets/d_7
../../../storage/parquets/d_8
../../../storage/parquets/d_9
../../../storage/parquets/d_a
../../../storage/parquets/d_b
../../../storage/parquets/d_c
../../../storage/parquets/d_d
../../../storage/parquets/d_e
../../../storage/parquets/d_f


In [36]:
import psutil
import time

def get_free_mem():
    mem_info = psutil.virtual_memory()
    free_mem = mem_info.available /mem_info.total
    return free_mem

def wait4ram(d):
    fm = get_free_mem()
    while fm < 0.2:
        print("free_ram:{}".format(fm))
        print("waiting:"+d["f"])
        
        time.sleep(1200)
        fm = get_free_mem()
        gc.collect()
    print("free_ram:{}".format(fm))
    print("start:"+d["f"])

In [29]:
import pyarrow as pa
import pyarrow.csv as csv

In [44]:
def csv2snp4vaex(d):
    wait4ram(d)
    
    chunk_size = int(1e+7)
    f = d["f"]
    f_out = d["f_pq"]
    f_out_base = d["f_pq_base"]
    sch = d["scheme"]
    tb = d["tbl"]
    ym = d["ym"]    
    odr = d["odr"]
    
    cols_drops = d_drop[(sch, tb)]
    
    f_d_id1n = d_dict[sch][ym].replace("strage", "share")
    d["d_id1"] = f_d_id1n
#     d_id1n = p2d(f_d_id1n)
    
    df_tmp = dd.read_csv(f, assume_missing=True, header=None, 
#                              low_memory=False, 
                     blocksize = "64MB",
#                      sample= int(2e+7),
                     sep=",",dtype="object")
    ncol = len(df_tmp.columns)
    d["ncol"] = ncol
    del df_tmp
    
    cols_drop_ind = [int(c[1:]) -1 for c in cols_drops]
    need_cols = [c for c in range(ncol) if c not in cols_drop_ind]
    need_cols_name = ["c" +str(i+1) for i in need_cols]
    
    #cols_drops = ["c" + str(int(c[1:])) for c in cols_drops]
    
    d_proc_meta = d.copy()

#     f_out = "../../../work/procdat/insvaex/d_xxx/in_{0}_{1}_{2}_{3}_xxx_sxyyy_{4}.hdf5".format(ym, sch, tb, odr, ts)
    f_whole = "../../../storage/parquets/in_{0}_{1}_{2}_{3}.parquet".format(ym, sch, tb, odr)
#     f_meta = f_out.replace(".hdf5", "_meta.json").replace("insvaex/d_xxx","metajsons")
    
    d_proc_meta["need_cols"] = need_cols
    d_proc_meta["need_cols_name"] = need_cols_name
    d_proc_meta["f_out"] = f_out
#     d_proc_meta["f_meta"] = f_meta
    d_proc_meta["f_whole_hdf5"] = f_whole
    d_proc_meta["f"] = f
    d_proc_meta["ts"] = ts
    d_proc_meta["cols_drops"] = cols_drops
    d_proc_meta["f_out_chr"] = dict()
    
    for c in chrs:
        d_proc_meta["f_out_chr"][c] = f_out.replace("xxx", c)

#     d2j(d=d_proc_meta,
#         f=f_meta)
    vaex.cache.off()
    vaex.read_csv(f, convert=f_whole, header=None,
              chunk_size = int(1e+6),
                  dtype="str"
             )
    

    
#     dfs = open(f_whole)
#     dfs = vaex.read_csv(f, convert=f_whole, header=None,
# #               chunk_size = int(1e+6),
#                   dtype="str"
#              )

    

#     for cx in range(ncol):
#         dfs.rename(str(cx), "c" + str(cx+1))
    
#     dfs.drop(columns = cols_drops, inplace = True)
    

#     dfs["id1n"] = dfs["c2"].apply(lambda x:d_id1n[x])
#     dfs["cap"] = dfs["id1n"].str.get(0)
#     dfs.materialize(inplace = True)
    
#     return dfs

#     total_row = len(dfs)
#     j = 0
#     for start in range(0, total_row, chunk_size):
#         print(start)
#         end = min(start + chunk_size, total_row)
#         df= dfs[start:end]

#         gc.collect()
        
#         #df = df.materialize()
                
#         for capx, df_sub in df.groupby("cap"):
#             cx = capx[0]
#             df_sub.drop(columns=["cap"], inplace=True)
#             f_out_sub = d_proc_meta["f_out_chr"][cx].replace("yyy", format(j, '08'))

#             print(f_out_sub)
#             df_sub.export_hdf5(f_out_sub)
#         del df
#         j = j +1
#         gc.collect()
        
#     del dfs
#     gc.collect()
    return d_proc_meta



#     dfs =vaex.from_csv_arrow(f,
#                             read_options=csv.ReadOptions(autogenerate_column_names =True,
#                                                          block_size=chunk_size))


#     return dfs
#     dfs =  vaex.from_csv(f,
#                    header=None, 
# #                         convert=f_whole,
# #                      low_memory=False, 
# #                      sep=",",
#                      usecols = need_cols
#                      ,dtype="str",
#                   chunk_size = chunk_size
#              )
#     dfs = vaex.open(f_whole,
#                     sep=",",
#                      usecols = need_cols
#                      ,dtype="str",
# #                   chunk_size = chunk_size)

#     for j, df in enumerate(dfs):
#         for c0, c1 in zip(need_cols, need_cols_name):
#             df.rename(str(c0), c1)
#         df["id1n"] = df["c2"].apply(lambda x:d_id1n[x])
#         df["cap"] = df["id1n"].str.get(0)
#         df.materialize(inplace = True)
#         gc.collect()
        
#         #df = df.materialize()
                
#         for capx, df_sub in df.groupby("cap"):
#             cx = capx[0]
#             df_sub.drop(columns=["cap"], inplace=True)
#             f_out_sub = d_proc_meta["f_out_chr"][cx].replace("yyy", format(j, '08'))

#             print(f_out_sub)
#             df_sub.export_hdf5(f_out_sub)
#         del df
#         gc.collect()
#     print(f_out)
#     del dfs
#     gc.collect()
#     return d_proc_meta

In [45]:
# l_proc[:4]

In [46]:
# tes = csv2snp4vaex(l_proc[0])

In [47]:
# tes


In [48]:
l_proc[:3]

[{'f': '../../../share3/hdw050/202209-018/data/pha/re/pha_re_201806_002.csv',
  'gp0': 't/hdw050/202209-018/data/pha/re/pha_re_201806_002.csv',
  'gp': 't/hdw0',
  'file': 'pha_re_201806_002.csv',
  'scheme': 'pha',
  'tbl': 're',
  'ym': '201806',
  'odr': '002',
  'f_pq': '../../../work/procdat/insvaex/pha_re_201806_002.hdf5',
  'f_pq_base': 'pha_re_201806_002.hdf5'},
 {'f': '../../../share3/hdw051/202209-018/data/med/sy/med_sy_201905_002.csv',
  'gp0': 't/hdw051/202209-018/data/med/sy/med_sy_201905_002.csv',
  'gp': 't/hdw0',
  'file': 'med_sy_201905_002.csv',
  'scheme': 'med',
  'tbl': 'sy',
  'ym': '201905',
  'odr': '002',
  'f_pq': '../../../work/procdat/insvaex/med_sy_201905_002.hdf5',
  'f_pq_base': 'med_sy_201905_002.hdf5'},
 {'f': '../../../share3/hdw051/202209-018/data/med/si/med_si_201209_017.csv',
  'gp0': 't/hdw051/202209-018/data/med/si/med_si_201209_017.csv',
  'gp': 't/hdw0',
  'file': 'med_si_201209_017.csv',
  'scheme': 'med',
  'tbl': 'si',
  'ym': '201209',
  'od

In [49]:
1

1

In [50]:
import random
random.seed(123)
random.shuffle(l_proc)

In [51]:
%%time
timeoutx = 1e+8
result = joblib.Parallel(n_jobs=16, verbose = 10, timeout=timeoutx,
#                          backend="multiprocessing",
                        )(joblib.delayed(csv2snp4vaex)(d) for i, d in enumerate(l_proc))

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
/opt/conda/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:  3.8min
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:  5.7min
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:  7.5min
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:  9.8min
[Parallel(n_jobs=16)]: Done  53 tasks      | elapsed: 13.1min
[Parallel(n_jobs=16)]: Done  66 tasks      | elapsed: 16.1min
[Parallel(n_jobs=16)]: Done  81 tasks      | elapsed: 20.4min
[Parallel(n_jobs=16)]: Done  96 tasks      | elapsed: 23.3min
[Parallel(n_jobs=16)]: Done 113 tasks      | elapsed: 26.1min
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed: 28.8min
[Parallel(n_jobs=16)]: Done 149 tasks      | elapsed: 32

CPU times: user 40.5 s, sys: 11.4 s, total: 51.8 s
Wall time: 1d 12h 27s


[Parallel(n_jobs=16)]: Done 10017 out of 10017 | elapsed: 2160.5min finished


In [52]:
d2p(d=result, f="./vaex_dict_res_{}.pkl".format(ts))

0

In [53]:
ts

'20231003_083542'

In [54]:
dfx = pd.DataFrame().from_dict(result)

In [55]:
dfx.to_pickle("./vaex_dict_df_{}.pkl".format(ts))

In [ ]:
# df_proc = 